###  Instalar o PyTorch

* Com GPU (CUDA): Verifique sua versão de CUDA (ex.: 11.7) e use o comando fornecido no site oficial do PyTorch:

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


* Sem GPU: Para instalar uma versão que roda na CPU:

In [ ]:
pip install torch torchvision torchaudio


### init

In [ ]:
pip install transformers datasets torchmetrics

### (Opcional) Instalar CUDA Toolkit

In [ ]:
pip install torch torchvision torchaudio  # Escolha a versão CUDA ou CPU
pip install transformers
pip install datasets torchmetrics         # (opcional, para datasets e métricas)


* Após a instalação, verifique se o PyTorch está funcionando corretamente:

```python
import torch
print(torch.cuda.is_available())  # Deve retornar True se CUDA estiver configurado

```

#### Pytorch

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# 1. Preparação do Tokenizer e Modelo Pré-treinado
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# 2. Configuração do Dataset
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        return tokens.input_ids.squeeze(), tokens.attention_mask.squeeze()

In [ ]:
# Dados de exemplo
texts = ["Exemplo de texto para treinamento.", "Outra linha de texto."]
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# 3. Configuração do Otimizador e Função de Custo
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 4. Loop de Treinamento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for input_ids, attention_mask in dataloader:
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        
        # Forward
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")